# <center> Statistical Control Charts
## <center> Systems Engineering and Analysis
## <center> <img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="400" /> 
### <center> Prepared by: Dr. Jeremy Daily

This notebook demonstrates a Monte Carlo method for constructing the $\overline{X}$ and $R$ Charts.

In [1]:
from scipy.stats import uniform, norm
import numpy as np
from math import sqrt

In [2]:
# Generate an example process
m = 100000 #the number of rows in Table 11.1. If this number is big, we'll get convergence.
n = 6 # The number of columns in Table 11.1. This is the sample size.
N = m*n
location = 20 #mean
scale = 10 #standard deviation
process_samples = norm.rvs(location,scale,size=N)

In [3]:
# This should match the theoretical mean
np.mean(process_samples)

19.984186431091555

In [4]:
mean, var, skew, kurt = norm.stats(location,scale,moments='mvsk')
print(f"Theoretical moments: \n mean = {mean}\n var = {var}\n skewness = {skew}\n kurtosis = {kurt}")

Theoretical moments: 
 mean = 20.0
 var = 100.0
 skewness = 0.0
 kurtosis = 0.0


In [5]:
#Estimate the process mean
x_bar = np.mean(process_samples[:n])
x_bar

25.481706997023498

In [6]:
R = max(process_samples[:n])-min(process_samples[:n])
R

19.085697065438932

In [7]:
#Calculate Eq 11.1 and 11.2
sum_x_bar = 0
sum_R = 0
for i in range(m):
    x_bar = np.mean(process_samples[n*i:n*(i+1)]) 
    sum_x_bar += x_bar
    R = max(process_samples[n*i:n*(i+1)])-min(process_samples[n*i:n*(i+1)])
    sum_R += R
x_bar_bar = sum_x_bar/m #Eq 11.1
print(f"x_bar_bar = {x_bar_bar}")
R_bar = sum_R/m #Eq 11.2
print(f"R_bar = {R_bar}")

x_bar_bar = 19.98418643109148
R_bar = 25.33399063248031


In [8]:
#compare to overall average. The overall average is x_bar_bar
np.mean(process_samples)

19.984186431091555

In [9]:
#Estimate the standard deviation of the overall process samples
std=np.std(process_samples,ddof=1)
std

9.996996619619495

In [10]:
# Variance is the standard deviation squared
std**2

99.9399414126836

In [11]:
n

6

In [12]:
#Simulated result for d2
d2 = R_bar/std
d2

2.534160167940976

In [13]:
#Theoretical variance
R_bar/sqrt(var)

2.533399063248031

The table value for n=6 is 2.534. This is just one example. You can change the value of n at the beginning and run all the cells again. Or, you can put the equations in a loop and iterate through the different values of n.

In [14]:
# Calculate d2 in table 11.2 using a Monte Carlo Method
#nlist = [n for n in range(2,11)]
print("Table 11.2: Factors for the Construction of Xbar and R Charts")
print("n\td2\tA2\tD3\tD4")
print("---\t---\t---\t---\t---")
N = m*10 #Use the previous m and calculate the overall number of samples for n=10
process_samples = norm.rvs(location,scale,size=N)
std=np.std(process_samples,ddof=1)
#Iterate through the n values.
for n in range(2,11):
    #Calculate the R_bar statistics, since it depends on the sample size, n
    sum_R = 0
    sum_R_squared = 0
    for i in range(m):
        R = max(process_samples[n*i:n*(i+1)])-min(process_samples[n*i:n*(i+1)])
        sum_R += R
        sum_R_squared += R**2
    R_bar = sum_R/m #Eq 11.2
    R_std = sqrt(m*sum_R_squared - sum_R**2)/m #Standard deviation of R
    
    d2 = R_bar/std #manipulate Eq 11.3
    A2 = 3/(d2*sqrt(n)) #Close to Eq 11.4
    
    # Lower range bounds can't be negative. 
    D3 = max(0,(R_bar - 3*R_std)/R_bar) #Lower bound from manipulating Eq 11.8
    D4 = (R_bar + 3*R_std)/R_bar #Upper bound from manipulating Eq 11.7
    
    print("{}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}".format(n,d2,A2,D3,D4))

Table 11.2: Factors for the Construction of Xbar and R Charts
n	d2	A2	D3	D4
---	---	---	---	---
2	1.133	1.872	0.000	3.266
3	1.694	1.023	0.000	2.577
4	2.061	0.728	0.000	2.280
5	2.324	0.577	0.000	2.115
6	2.535	0.483	0.000	2.007
7	2.705	0.419	0.074	1.926
8	2.848	0.372	0.135	1.865
9	2.971	0.337	0.184	1.816
10	3.077	0.308	0.223	1.777


This table, calculated with the Monte Carlo simulation method, should match at least 3 digits on each entry of Table 11.2.